In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter,defaultdict
from datetime import datetime

In [ ]:
dir_path = './'

df_1 = pd.read_csv(dir_path + '01-Tweets_user_createdAt_place.csv')
df_2 = pd.read_csv(dir_path + '02-Tweets_type.csv')
print('Merge 1 - start')
df_merge = df_1.merge(df_2, on='tweet_id')
print('Merge 1 - end')
df_merge.to_csv(dir_path + 'merge-01-02.csv', index=False)
print('Merge 1 - saved')
del df_merge
del df_2
df_3 = pd.read_csv(dir_path + '03-Tweets_media_url_contributors_mentions.csv')
print('Merge 2 - start')
df_merge = df_1.merge(df_3, on='tweet_id')
print('Merge 2 - end')
df_merge.to_csv(dir_path + 'merge-01-03.csv', index=False)
print('Merge 2 - saved')
del df_3
del df_merge

df_4 = pd.read_csv(dir_path + '04-Tweets_hashtags.csv')
print('Merge 3 - start')
df_merge = df_1.merge(df_4, on='tweet_id')
print('Merge 3 - end')
df_merge.to_csv(dir_path + 'merge-01-04.csv', index=False)
print('Merge 3 - saved')
del df_4
del df_merge

In [ ]:
chunksize = 500_000

with_places = pd.DataFrame()
users = pd.DataFrame()

for chunk in tqdm(pd.read_csv(dir_path + '01-Tweets_user_createdAt_place.csv', chunksize=chunksize)):

    chunk['createdAt'] = pd.to_numeric(chunk['createdAt'], errors='coerce')

    chunk['createdAt'] = datetime.fromtimestamp(int(chunk['createdAt'].values[0]) / 1000).strftime('%Y-%m-%d')
    

    chunk['tweetId'] = [int(x,32) for x in chunk['tweetId']]
    chunk['userId'] = [int(x,32) for x in chunk['userId']]

    users = pd.concat([users,chunk.groupby(['userId','createdAt'])[['tweetId']].count().reset_index()])

    with_places = pd.concat([with_places,chunk[~chunk['place_fullName'].isna() & chunk['place_fullName'].str.len() > 1].groupby('createdAt')[['place_fullName']].count().reset_index()])
        
aa = users.groupby(['userId','createdAt'])[['tweetId']].sum()
aa.to_pickle(dir_path + '__users_day_stats.pickle')

In [ ]:
chunksize = 1_000_000
stats = defaultdict(Counter)

for chunk in tqdm(pd.read_csv(dir_path + 'merge-01-02.csv', chunksize=chunksize)):

    if 'createdAt' not in chunk.columns:
        chunk['createdAt'] = [int(x,36) for x in chunk['created_at']]

    chunk['createdAt'] = pd.to_numeric(chunk['createdAt'], errors='coerce') # asumo que le agregaste la columna
    chunk = chunk[~chunk['createdAt'].isna()].copy()
    chunk['createdAt'] = datetime.fromtimestamp(int(chunk['createdAt'].values[0]) / 1000).strftime('%Y-%m-%d')
    chunk['all'] = 1
    dd = chunk.groupby('createdAt')[['original','retweet','reply','quote','all']].apply(lambda x: (x > 0).sum()).to_dict(orient='index')
    
    for k,v in dd.items():
        stats[k].update(v)

pd.DataFrame(stats).to_csv(dir_path + '__tweet_types_per_day.csv')

In [ ]:
chunksize = 1_000_000

stats_count = defaultdict(Counter)
stats_freq = defaultdict(Counter)

for chunk in tqdm(pd.read_csv(dir_path + 'merge-01-03.csv', chunksize=chunksize)):

    if 'createdAt' not in chunk.columns:
        chunk['createdAt'] = [int(x,36) for x in chunk['created_at']]

    chunk['createdAt'] = pd.to_numeric(chunk['createdAt'], errors='coerce') # asumo que le agregaste la columna
    chunk = chunk[~chunk['createdAt'].isna()].copy()
    chunk['createdAt'] = datetime.fromtimestamp(int(chunk['createdAt'].values[0]) / 1000).strftime('%Y-%m-%d')
    chunk['all'] = 1

    dd = chunk.groupby('createdAt')[['media','url','contributors','mentions','all']].apply(lambda x: (x > 0).sum()).to_dict(orient='index') 
    for k,v in dd.items():
        stats_count[k].update(v)

    dd = chunk.groupby('createdAt')[['media','url','contributors','mentions','all']].sum().to_dict(orient='index') 
    for k,v in dd.items():
        stats_freq[k].update(v)

pd.DataFrame(stats_count).to_csv(dir_path + '__stats_content_tweets_per_dia.csv')
pd.DataFrame(stats_freq).to_csv(dir_path + '__stats_freq_content_tweets_per_dia.csv')

In [ ]:
chunksize = 1_000_000

hash_freq = defaultdict(Counter)

with_hash_day = defaultdict(Counter)

for chunk in tqdm(pd.read_csv(dir_path + 'merge-01-04.csv', chunksize=chunksize)):
    
    if 'createdAt' not in chunk.columns:
        chunk['createdAt'] = [int(x,36) for x in chunk['created_at']]

    chunk['createdAt'] = pd.to_numeric(chunk['createdAt'], errors='coerce') # asumo que le agregaste la columna
    chunk = chunk[~chunk['createdAt'].isna()].copy()
    chunk['createdAt'] = datetime.fromtimestamp(int(chunk['createdAt'].values[0]) / 1000).strftime('%Y-%m-%d')
    chunk['all'] = 1

    # hashtag frequency by date
    aa = chunk[~chunk['hashtags'].isna()].groupby('createdAt')[['hashtags']].agg(lambda x: ''.join(x)).to_dict(orient='index')
    for date,hh in aa.items():
        hash_freq[date].update(hh['hashtags'].split(' '))

    aa = chunk[~chunk['hashtags'].isna()].groupby('createdAt')[['hashtags']].count().to_dict(orient='index')
    for date,hh in aa.items():
        with_hash_day[date].update(hh)

pd.DataFrame(hash_freq).to_csv(dir_path + '__stats_hash_freq.csv')
pd.DataFrame(with_hash_day).to_csv(dir_path + '__stats_with_hash_day.csv')

In [ ]:
chunksize = 1_000_000

hash_per_tweet= pd.DataFrame()

for chunk in tqdm(pd.read_csv(dir_path + 'merge-01-04.csv', chunksize=chunksize)):
    chunk['createdAt'] = pd.to_numeric(chunk['createdAt'], errors='coerce') # asumo que le agregaste la columna
    chunk = chunk[~chunk['createdAt'].isna()].copy()
    chunk['createdAt'] = datetime.fromtimestamp(int(chunk['createdAt'].values[0]) / 1000).strftime('%Y-%m-%d')
    chunk = chunk[~chunk['hashtags'].isna()]
    chunk['hashtags'] = chunk['hashtags'].apply(lambda x: len(x.split(' ')))
    aa = chunk.groupby('createdAt')[['hashtags']].agg(['count','sum']).reset_index()
    hash_per_tweet = pd.concat([hash_per_tweet,aa])
    

hash_per_tweet.to_csv(dir_path + 'hash_per_tweet.csv', index=False)
hash_per_tweet

In [ ]:
def add_temporal_information(df,col=None):

    if col is None:
        df['date'] = pd.to_datetime(df.index)
    else:
        df['date'] = pd.to_datetime(df[col])
        
    df['month'] = df['date'].dt.year.astype(str) + '-' + df['date'].dt.month.astype(str)

    df['week_of_year'] = df['date'].dt.year.astype(str) + '-' + df['date'].dt.isocalendar().week.astype(str)

    df = df.drop(columns=['date'])

    return df

In [ ]:
# USERS
df_users_day_stats = pd.read_pickle(dir_path + '__users_day_stats.pickle')
df_users_day_stats = df_users_day_stats.reset_index()
df_users_day_stats = add_temporal_information(df_users_day_stats,col='createdAt')
df_users_day_stats

In [ ]:
aa = df_users_day_stats.groupby('userId')[['tweetId']].sum()
aa.reset_index().to_pickle(dir_path + '__tweets_user_all.pickle')
aa

In [ ]:
bb = df_users_day_stats.groupby(['userId','week_of_year'])[['tweetId']].sum()
bb.reset_index().to_pickle(dir_path + '__tweets_user_week.pickle')
bb

In [ ]:
cc = df_users_day_stats.groupby(['userId','month'])[['tweetId']].sum()
cc.reset_index().to_pickle(dir_path + '__tweets_user_month.pickle')
cc